This notebook demonstrates the usage of the resilience/socio-economic capacity model.

# Loading functions

We first import function librairies. The first ones are standard python librairies.

In [40]:
#The main package we use. Manipulates Excel-like tables (called dataframes) with named rows and columns
import pandas as pd 

In [41]:
#ploting capacities
import matplotlib.pyplot as plt 
#make plots appear in this notebook
%matplotlib inline  

#Default options for plots: this controls the font used in figures
font = {'family' : 'sans serif',
    'size'   : 22}
plt.rc('font', **font)


# Loading and exploring all the data

Here we read the whole data set we need into a pandas table (called DataFrame).
This specific dataset was put together from a variety of different sources using another python notebook: [data_gathering_demo.ipynb](data_gathering_demo.ipynb). In the future we could build directly an excel file with all the data the model needs.

In [55]:
#Load the excel data file into a dataframe (table)
df=pd.read_excel("all_data_compiled.xlsx", 
                 index_col=0, #tells Pandas to use the column "Province" as the index of the table
                 skiprows=[0, 2] #The first (0) row is a description of the variable so we skip it, and the 3 (2) row is empty
                ) 

#Displays the first rows of that dataframe
df.head()

,gdp_pc_pp,pop,plgp,unemp,bashs,pov_head,shew,ophe,gdp_pc_pp_nat,rel_gdp_pp,...,pv,v_s,protection,avg_prod_k,T_rebuild_K,pi,nat_buyout,sigma_r,sigma_p,income_elast
Abra,133.688,240135.244121,0.8827,0.165,0.857720,0.373595,0.953416,0.031414,184.136685,0.726026,...,0.234093,0.100000,10.57,0.23,3,0.2,0.3,0.333333,0.333333,1.5
Agusan Del Norte,179.014,661728.454375,0.7075,0.210,0.921445,0.346715,0.821277,0.034563,184.136685,0.972180,...,0.911730,0.100000,9.41,0.23,3,0.2,0.3,0.333333,0.333333,1.5
Agusan Del Sur,126.492,677779.682154,0.6871,0.210,0.727442,0.480785,0.821277,0.034563,184.136685,0.686946,...,0.501310,0.100000,8.61,0.23,3,0.2,0.3,0.333333,0.333333,1.5
Aklan,119.962,554414.442422,0.7210,0.188,0.806176,0.249662,0.823003,0.044318,184.136685,0.651483,...,NaN,0.135089,0.00,0.23,3,0.2,0.3,0.333333,0.333333,1.5
Albay,158.629,1264097.894966,0.7930,0.346,0.840840,0.409587,0.800000,0.032568,184.136685,0.861474,...,NaN,0.100000,0.00,0.23,3,0.2,0.3,0.333333,0.333333,1.5


The columns names correspond to variable in a model, so they are not all self-explanatory. For a description of the variables, see [inputs/inputs_info.csv](inputs/inputs_info.csv)

Let's get familiar with Dataframes. Here is how to look at specific rows or columns.

In [ ]:
df.ix["Abra"].head() #all the data we have on Abra. The head() method returns only the first few lines for saving space

In [ ]:
df["pop"].head() #population in all provinces

Rather than displaying the actual numbers, we can plot it in a few ways. Here is the distribution of income accross provinces

In [ ]:
df.gdp_pc_pp.plot.hist(figsize=(7,7))
plt.xlabel("Average income in the province ($\\times$1000 pesos)");

And below is a scatter plot showing average income, poverty incidence, and size (population) of each province

In [ ]:
df.plot.scatter(x="gdp_pc_pp", y="pov_head", s=df["pop"]/1e4, alpha=0.5, figsize=(7,7))
plt.xlabel("Average income")
plt.ylabel("Poverty incidence")
plt.ylim(0);

# Computing socio-economic capacity

Now that we have minimum familiarity with dataframes, let us use the resilience model. Let us first read the function definitions, from [res_ind_lib.py](res_ind_lib.py)

In [ ]:
from res_ind_lib import *     #MAIN library: the functions used to compute risk, resilience, etc

Before running the model, for technical reasons, we need to define "reference" values for some variables. The following function does it.

<small>(The technical reason is that within the function that computes resilience, some variables -- exposure fa, exposure bias pe, and poverty headcount pov_head -- are used to compute the vulnerabilities of poor and nonpoor people (vp, vr) from average vulnerability and vulnerability bias (v, pv). Making copies of this variables now, faref=fa, peref=pe, etc., ensures that policy experiments that modify these variables, for instance exposure bias, leaves vp and vr unchanged).</small>

In [ ]:
df = def_ref_values(df).dropna()
df[["fa", "faref"]].head()

In [ ]:
df = def_ref_values(df).dropna()
df[["fa", "faref"]].head()

#### Central cases

In [ ]:
df_with_results=compute_resiliences(df)


print("nb Provinces with all data :"+str(df.dropna().shape[0]))
nb_countries_all_results = df_with_results["resilience"].dropna().shape[0]
print("nb Provinces with all data and results :"+str(nb_countries_all_results))

df_with_results.to_csv("all_data_and_results.csv")


#### Basic reporting

In [ ]:
#Some stats
a=df_with_results.resilience;
a.min()*100,a.mean()*100,a.max()*100
print("Resilience averages {mean:.0%} across our sample, ranging from {min:.0%} to {max:.0%}".format(
    min=a.min(),mean=a.mean(),max=a.max()))
(100*a).plot.hist(figsize=(7,7));

In [ ]:
#Outputs table with main results
to_output = ['gdp_pc_pp', 'pop',"protection","fa","v","resilience","risk"]
a=df_with_results[to_output].copy()
a.loc[:,["fa","v","resilience","risk"]]=100*a[["fa","v","resilience","risk"]]
desc=pd.read_csv("inputs/inputs_info.csv").set_index('key')["descriptor"]
a=a.rename(columns=desc).dropna()
a.to_excel("main results.xlsx")
a.head()

# Categorizing provinces

In [ ]:
categories = pd.DataFrame([make_tiers(df_with_results.resilience), make_tiers(df_with_results.risk)]).T.dropna()
categories_to_formated_excel_file(categories,"categories.xlsx")
categories.head()

Risk to assets, socio-economic capacity, and total welfare risk can also be visualized in maps (see [draw_maps.ipynb](draw_maps.ipynb) for how to build these maps)

#### Mapping the results

 <table border="0" style="border:none">
  <tr>
    <td>![title](img/map_and_legend_of_asset_risk.png)</td>
    <td>![title](img/map_and_legend_of_se_capacity.png)</td>
    <td>![title](img/map_and_legend_of_welfare_risk.png)</td>
  </tr>
</table> 


# Policy experiments

#### Impact of decreasing poverty incidence

In [ ]:
df_share1 = compute_resiliences( df.dropna().assign(pov_head=df.pov_head-1/100) )

#welfare losses
wa=df_with_results.total_equivalent_cost.sum()*1e-9
wb=df_share1.total_equivalent_cost.sum()*1e-9

#asset losses
ka = (df_with_results.dKtot/df.protection).sum()*1e-9
kb = (df_share1.dKtot/df_share1.protection).sum()*1e-9

print(" Decreasing poverty incidence by 1 percent in the {nb} provincies \
would increase asset losses from USD {ka:0.0f} to {kb:0.0f} billions per year \
(+{XX:.1f} percent), but reduce welfare losses by {YY:.2} percent".format(
        XX=100*(kb-ka)/ka,ka=ka,kb=kb,YY=100*(wa-wb)/wa,nb=nb_countries_all_results)
     )

#### Impact of increasing effective scale up

In [ ]:
df_su = compute_resiliences( df.dropna().assign(sigma_p=1) )

#welfare losses
wa=df_with_results.total_equivalent_cost.sum()*1e-6
wb=df_su.total_equivalent_cost.sum()*1e-6

#asset losses
ka = (df_with_results.dKtot/df.protection).sum()*1e-6
kb = (df_su.dKtot/df_su.protection).sum()*1e-6

print(" Increasing effective scale up to 100% in the {nb} provincies \
would increase asset losses from {ka:0.2f} to {kb:0.2f} billions per year \
(+{XX:.1f} percent), but reduce welfare losses by {YY:.2} percent".format(
        XX=100*(kb-ka)/ka,ka=ka,kb=kb,YY=100*(wa-wb)/wa,nb=nb_countries_all_results)
     )